# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
%load_ext dotenv
%dotenv

In [2]:
import os

In [3]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
from glob import glob
# Get PRICE_DATA path
PRICE_DATA = os.getenv("PRICE_DATA")

In [5]:
print("PRICE_DATA:", PRICE_DATA)

PRICE_DATA: ../../05_src/data/prices/


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [6]:
# List parquet files
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)
print("Found parquet files:", parquet_files)

Found parquet files: ['../../05_src/data/prices\\AAXN\\AAXN_2001\\part.0.parquet', '../../05_src/data/prices\\AAXN\\AAXN_2001\\part.1.parquet', '../../05_src/data/prices\\AAXN\\AAXN_2002\\part.0.parquet', '../../05_src/data/prices\\AAXN\\AAXN_2002\\part.1.parquet', '../../05_src/data/prices\\AAXN\\AAXN_2003\\part.0.parquet', '../../05_src/data/prices\\AAXN\\AAXN_2003\\part.1.parquet', '../../05_src/data/prices\\AAXN\\AAXN_2004\\part.0.parquet', '../../05_src/data/prices\\AAXN\\AAXN_2004\\part.1.parquet', '../../05_src/data/prices\\AAXN\\AAXN_2005\\part.0.parquet', '../../05_src/data/prices\\AAXN\\AAXN_2005\\part.1.parquet', '../../05_src/data/prices\\AAXN\\AAXN_2006\\part.0.parquet', '../../05_src/data/prices\\AAXN\\AAXN_2006\\part.1.parquet', '../../05_src/data/prices\\AAXN\\AAXN_2007\\part.0.parquet', '../../05_src/data/prices\\AAXN\\AAXN_2007\\part.1.parquet', '../../05_src/data/prices\\AAXN\\AAXN_2008\\part.0.parquet', '../../05_src/data/prices\\AAXN\\AAXN_2008\\part.1.parquet', '.

In [7]:
# Load Parquet into Dask
dd_feat = dd.read_parquet(parquet_files, engine="pyarrow")

# Put all rows of each ticker in the same partition
dd_feat = dd_feat.set_index("ticker").persist()

# Lag features (done lazily in Dask)
dd_feat["Close_lag_1"] = dd_feat.groupby("ticker", group_keys=False)["Close"].shift(1)
dd_feat["Adj_Close_lag_1"] = dd_feat.groupby("ticker", group_keys=False)["Adj Close"].shift(1)

# Returns & daily range
dd_feat["returns"] = (dd_feat["Close"] / dd_feat["Close_lag_1"]) - 1
dd_feat["hi_lo_range"] = dd_feat["High"] - dd_feat["Low"]

C:\Users\User\AppData\Local\Temp\ipykernel_9860\745491132.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat["Close_lag_1"] = dd_feat.groupby("ticker", group_keys=False)["Close"].shift(1)
C:\Users\User\AppData\Local\Temp\ipykernel_9860\745491132.py:9: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat["Adj_Close_lag_1"] = dd_feat.groupby("ticker", group_keys=False)["Adj Close"].shift(1)


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [8]:
# Convert Dask DataFrame to pandas for easier operations
dd_feat = dd_feat.compute()

In [9]:
# Rolling mean of returns over 10 days per ticker
dd_feat["returns_ma_10"] = (
    dd_feat.groupby("ticker")["returns"]
           .transform(lambda x: x.rolling(window=10, min_periods=1).mean())
)

print(dd_feat.head(15))


             Date      Open      High       Low     Close  Adj Close  \
ticker                                                                 
AAXN   2001-06-08  0.566770  0.566770  0.562570  0.562570   0.562570   
AAXN   2001-06-11  0.566770  0.579080  0.566770  0.570880   0.570880   
AAXN   2001-06-13  0.558000  0.571000  0.555000  0.558000   0.558000   
AAXN   2001-06-19  0.508333  0.508333  0.460000  0.460000   0.460000   
AAXN   2001-06-20  0.458333  0.458333  0.445833  0.445833   0.445833   
AAXN   2001-06-21  0.445833  0.445833  0.445833  0.445833   0.445833   
AAXN   2001-06-22  0.445833  0.479167  0.445833  0.479167   0.479167   
AAXN   2001-06-25  0.479167  0.483333  0.475000  0.479167   0.479167   
AAXN   2001-06-26  0.475000  0.520000  0.475000  0.520000   0.520000   
AAXN   2001-06-27  0.504167  0.508333  0.504167  0.508333   0.508333   
AAXN   2001-06-28  0.511667  0.511667  0.511667  0.511667   0.511667   
AAXN   2001-06-29  0.516667  0.540833  0.516667  0.533333   0.53

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

Yes, because Dask does not provide an easy way to do rolling windows with groupby.

+ Would it have been better to do it in Dask? Why?

For very large datasets that do not fit into memory, using Dask would be more efficient because it can handle out-of-core computations and parallelize operations. However, for moderate-sized datasets, pandas is simpler, more convenient, and easier to debug.

(1 pt)


## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.